# DX 704 Week 3 Project

This week's project will give you practice with optimizing choices for bandit algorithms.
You will be given access to the bandit problem via a blackbox object, and you will investigate the bandit rewards to pick a suitable algorithm.

The full project description, a template notebook and supporting code are available on GitHub: [Project 3 Materials](https://github.com/bu-cds-dx704/dx704-project-03).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Pick a Bandit Algorithm

Experiment with the multi-armed bandit interface using seed 0 to learn about the distribution of rewards and decide what kind of bandit algorithm will be appropriate.
A histogram will likely be helpful.

In [1]:
# DO NOT CHANGE

%pip install numpy 

import numpy as np

class BanditProblem(object):
    def __init__(self, seed):
        self.seed = seed
        self.rng = np.random.default_rng(seed)

        self.num_arms = 3
        self.ns = self.rng.integers(low=1, high=10, size=self.num_arms)
        self.ps = self.rng.uniform(low=0.2, high=0.4, size=self.num_arms)

    def get_num_arms(self):
        return self.num_arms

    def get_reward(self, arm):
        if arm < 0 or arm >= self.num_arms:
            raise ValueError("Invalid arm")

        x = self.rng.uniform()
        x *= self.rng.binomial(self.ns[arm], self.ps[arm])

        return x



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
bandit0 = BanditProblem(0)

In [3]:
bandit0.get_num_arms()

3

In [ ]:
bandit0.get_reward(arm=0) 

1.8255111545554434

In [5]:
# YOUR CHANGES HERE

text = ("UCB1 (Upper Confidence Bound) — rewards are bounded, continuous-valued (not just 0/1), "
        "so UCB1 is appropriate for bounded stationary rewards, while Bernoulli Thompson Sampling "
        "doesn’t match the reward type and epsilon-greedy is less efficient because it explores uniformly "
        "rather than using confidence bounds.\n")

with open("algorithm-choice.txt", "w") as f:
    f.write(text) 

Based on your investigation, pick an appropriate bandit algorithm to implement from the algorithms covered this week.
Write a file "algorithm-choice.txt" that states your choice and give a single sentence justifying your choice and rejecting the alternatives.
Keep your explanation concise; you should be able to justify your choice solely based on the type of numbers observed, and whether those match the bandit algorithms that you have learned.

## Part 2: Implement Bandit

Based on your decision, implement an appropriate bandit algorithm and pick 1000 actions using seed 2026.

In [7]:
# YOUR CHANGES HERE

import math
import numpy as np

class UCB1Agent:
    def __init__(self, n_arms, reward_bound=9.0):
        self.n_arms = n_arms
        self.B = float(reward_bound)
        self.counts = np.zeros(n_arms, dtype=int)
        self.sums = np.zeros(n_arms, dtype=float)  # normalized reward sums

    def select_arm(self):
        for i in range(self.n_arms):
            if self.counts[i] == 0:
                return i

        t = int(self.counts.sum())
        best_i = 0
        best_ucb = -float("inf")

        for i in range(self.n_arms):
            avg = self.sums[i] / self.counts[i]
            bonus = math.sqrt(2.0 * math.log(t) / self.counts[i])
            ucb = avg + bonus
            if ucb > best_ucb:
                best_ucb = ucb
                best_i = i

        return best_i

    def update(self, arm, reward):
        r = float(reward) / self.B  # normalize to [0,1]
        self.counts[arm] += 1
        self.sums[arm] += r


# --- run 1000 actions ---
bandit = BanditProblem(seed=2026)
K = bandit.get_num_arms()
agent = UCB1Agent(n_arms=K, reward_bound=9.0)

T = 1000
history_actions = []
history_rewards = []

for _ in range(T):
    a = agent.select_arm()
    r = bandit.get_reward(a)
    agent.update(a, r)
    history_actions.append(a)
    history_rewards.append(r) 

Write a file "history.tsv" with columns action and reward in the order that the actions were taken.

In [8]:
# YOUR CHANGES HERE

with open("history.tsv", "w") as f:
    f.write("action\treward\n")
    for a, r in zip(history_actions, history_rewards):
        f.write(f"{a}\t{r}\n") 

Submit "history.tsv" in Gradescope.

## Part 3: Action Statistics

Based on the data from part 2, estimate the expected reward for each arm and write a file "actions.tsv" with the columns action, min_reward, mean_reward, max_reward.

In [13]:
# YOUR CHANGES HERE

import math
import numpy as np

# --- UCB1 Agent ---
class UCB1Agent:
    def __init__(self, n_arms, reward_bound=9.0):
        self.n_arms = n_arms
        self.B = float(reward_bound)
        self.counts = np.zeros(n_arms, dtype=int)
        self.sums = np.zeros(n_arms, dtype=float)  # normalized reward sums

    def select_arm(self):
        for i in range(self.n_arms):
            if self.counts[i] == 0:
                return i

        t = int(self.counts.sum())
        best_i = 0
        best_ucb = -float("inf")

        for i in range(self.n_arms):
            avg = self.sums[i] / self.counts[i]
            bonus = math.sqrt(2.0 * math.log(t) / self.counts[i])
            ucb = avg + bonus
            if ucb > best_ucb:
                best_ucb = ucb
                best_i = i

        return best_i

    def update(self, arm, reward):
        r = float(reward) / self.B  # normalize to [0,1]
        self.counts[arm] += 1
        self.sums[arm] += r


# --- Run bandit for 1000 steps ---
bandit = BanditProblem(seed=2026)
K = bandit.get_num_arms()
agent = UCB1Agent(n_arms=K, reward_bound=9.0)

T = 1000
history_actions = []
history_rewards = []

for _ in range(T):
    a = agent.select_arm()
    r = bandit.get_reward(a)
    agent.update(a, r)
    history_actions.append(a)
    history_rewards.append(r)

# --- Write history.tsv ---
with open("history.tsv", "w") as f:
    f.write("action\treward\n")
    for a, r in zip(history_actions, history_rewards):
        f.write(f"{a}\t{r}\n")

# --- Compute per-arm stats ---
rewards_by_arm = {a: [] for a in range(K)}
for a, r in zip(history_actions, history_rewards):
    rewards_by_arm[a].append(r)

arm_stats = []
for a in range(K):
    arr = np.array(rewards_by_arm[a], dtype=float)
    min_r = float(arr.min()) if len(arr) > 0 else 0.0
    mean_r = float(arr.mean()) if len(arr) > 0 else 0.0
    max_r = float(arr.max()) if len(arr) > 0 else 0.0
    arm_stats.append((a, min_r, mean_r, max_r))

# --- Write actions.tsv ---
with open("actions.tsv", "w") as f:
    f.write("action\tmin_reward\tmean_reward\tmax_reward\n")
    for a, min_r, mean_r, max_r in arm_stats:
        f.write(f"{a}\t{min_r}\t{mean_r}\t{max_r}\n")

print("Done! Created history.tsv and actions.tsv") 

Done! Created history.tsv and actions.tsv


Submit "actions.tsv" in Gradescope.

## Part 4: Regret Estimates

Estimate the regret taking 1000 actions with the following strategies.

* uniform: Pick an arm uniformly at random.
* just-i: Always pick arm $i$. Do this for $i=0$ to $K-1$ where $K$ is the number of arms.
* actual: This should match your output in part 2.

These estimates should be based on your previous action statistics; you should not use the true action values from the bandit code.

In [14]:
# YOUR CHANGES HERE

import numpy as np

# ----- Inputs assumed to exist from earlier parts -----
# arm_stats: list of tuples (action, min_reward, mean_reward, max_reward)
# history_actions: list of actions taken in Part 2 (length 1000)

T = len(history_actions)

# Estimated mean reward for each arm (from Part 3)
means = {a: mean_r for (a, min_r, mean_r, max_r) in arm_stats}
K = len(means)

best_arm = max(means, key=lambda a: means[a])
best_mean = means[best_arm]

def regret_from_expected_reward(expected_reward_per_pull):
    # Regret over T pulls = T*(mu* - E[mu_strategy])
    return T * (best_mean - expected_reward_per_pull)

# 1) uniform: choose each arm with prob 1/K
uniform_expected = sum(means[a] for a in range(K)) / K
uniform_regret = regret_from_expected_reward(uniform_expected)

# 2) just-i: always choose arm i
just_regrets = []
for i in range(K):
    just_expected = means[i]
    just_regrets.append((f"just-{i}", regret_from_expected_reward(just_expected)))

# 3) actual: use the action frequencies from Part 2 (NOT the realized rewards)
counts = np.bincount(np.array(history_actions, dtype=int), minlength=K)
actual_expected = sum((counts[i] / T) * means[i] for i in range(K))
actual_regret = regret_from_expected_reward(actual_expected)

# Collect results
results = [("uniform", uniform_regret)] + just_regrets + [("actual", actual_regret)]

# Print nicely
print(f"Estimated best arm: {best_arm} with mean {best_mean:.6f}\n")
for name, reg in results:
    print(f"{name:8s}  regret={reg:.6f}")

# Write regret estimates to file (handy for Gradescope if they expect it)
with open("regret.tsv", "w") as f:
    f.write("strategy\tregret\n")
    for name, reg in results:
        f.write(f"{name}\t{reg}\n")

print("\nWrote regret.tsv") 

Estimated best arm: 0 with mean 1.240236

uniform   regret=688.536220
just-0    regret=0.000000
just-1    regret=949.552759
just-2    regret=1116.055901
actual    regret=405.709918

Wrote regret.tsv


Write your results to a file "strategies.tsv" with the columns strategy and regret.

In [15]:
# YOUR CHANGES HERE

with open("strategies.tsv", "w") as f:
    f.write("strategy\tregret\n")
    for name, reg in results:
        f.write(f"{name}\t{reg}\n")

print("Wrote strategies.tsv") 

Wrote strategies.tsv


Submit "strategies.tsv" in Gradescope.

## Part 5: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [16]:
with open("acknowledgments.txt", "w") as f:
    f.write(
        "I referenced course materials and example notebooks to review multi-armed bandit algorithms and used ChatGPT as a reference tool "
        "to sanity-check my understanding and help debug and refine my Python code. No outside collaborators were involved.\n"
    )

print("Wrote acknowledgments.txt")  

Wrote acknowledgments.txt


Submit "acknowledgments.txt" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.

Submit "project.ipynb" in Gradescope.